# Data Wrangling with Quandl

### Import the relevant Modules

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

In [1]:
import requests
import json

In [2]:
# Store the API - Get key by registering to http://www.quandl.com 
# Select the desired stock ticker, start date, and end date - save each parameter as a string
API_KEY = 'yvcucg9Ps58_Ta21JPEo'
STOCK_TICKER = 'AFX_X'
START_DATE = '2017-01-01'
END_DATE = '2017-12-31'

### Getting a glimpse at the data

In [3]:
# JSON structure will be returned
params = {'api-key': API_KEY}
req = requests.get(
    f'https://www.quandl.com/api/v3/datasets/FSE/{STOCK_TICKER}.json?&start_date=2021-09-01&end_date=2021-09-01', 
    params=params
)
req_json = req.json()
req_json

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-12-01T14:48:09.907Z',
  'newest_available_date': '2020-12-01',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2021-09-01',
  'end_date': '2020-12-01',
  'data': [],
  'collapse': None,
  'order': None,
  'database_id': 6129}}


### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 


In [4]:
# collect the stock exchange data using the start date, end date, and stock ticker
stock_params = {'api-key': API_KEY, 'start_date': START_DATE, 'end_date': END_DATE}
stock_req = requests.get(
    f'https://www.quandl.com/api/v3/datasets/FSE/{STOCK_TICKER}.json?', 
    params=stock_params
)

In [5]:
#Check Status Code (Status code of 200 means data is imported properly)
code = stock_req.status_code
if code == requests.codes.ok:
    print(f'{STOCK_TICKER} - Data properly imported')
else:
    print(f'Requests Error Code {code}')

AFX_X - Data properly imported


### 2. Convert the returned JSON object into a Python dictionary.

In [6]:
# used the standard python package 'json' to turn json object into a dict
stock_json = stock_req.json()
type(stock_json)

dict

### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [7]:
stock_json['dataset']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [8]:
open_idx = stock_json['dataset']['column_names'].index('Open')

In [9]:
# Create a list of all opening pricese over the time period
data_points = stock_json['dataset']['data']

opening = [x[open_idx] for x in data_points if x[open_idx] is not None]
opening

[51.76,
 51.65,
 51.45,
 51.05,
 51.16,
 51.88,
 52.73,
 52.37,
 52.7,
 53.11,
 52.64,
 52.29,
 52.28,
 51.5,
 50.89,
 50.8,
 51.21,
 49.5,
 49.52,
 48.64,
 49.64,
 49.09,
 49.13,
 49.11,
 48.8,
 48.4,
 47.25,
 46.57,
 47.03,
 47.09,
 47.98,
 48.4,
 48.38,
 47.3,
 47.65,
 46.42,
 46.16,
 45.81,
 45.0,
 45.88,
 46.29,
 46.53,
 45.48,
 45.2,
 45.01,
 45.16,
 44.9,
 45.08,
 45.72,
 46.01,
 45.8,
 45.61,
 45.5,
 45.58,
 45.97,
 45.64,
 46.2,
 46.19,
 46.01,
 45.36,
 44.51,
 43.58,
 42.0,
 42.35,
 42.3,
 42.3,
 41.48,
 42.29,
 42.54,
 42.65,
 42.5,
 42.29,
 42.35,
 42.49,
 43.21,
 42.81,
 42.7,
 43.0,
 42.66,
 43.0,
 42.38,
 42.16,
 42.0,
 42.0,
 41.71,
 42.11,
 42.64,
 42.72,
 42.82,
 42.46,
 42.42,
 42.28,
 41.88,
 42.4,
 42.53,
 42.12,
 41.3,
 41.73,
 43.5,
 44.9,
 45.85,
 45.13,
 45.34,
 45.25,
 45.24,
 44.94,
 45.26,
 45.16,
 44.91,
 44.7,
 45.31,
 45.57,
 45.74,
 45.06,
 45.5,
 45.6,
 45.07,
 44.67,
 44.29,
 44.94,
 44.64,
 44.79,
 45.5,
 44.67,
 45.83,
 45.29,
 45.01,
 45.73,
 46.68,

In [10]:
# Get high opening price
high_opening = max(opening)
high_opening

53.11

In [11]:
# Get low opening price
low_opening = min(opening)
low_opening

34.0

### 4. What was the largest change in any one day (based on High and Low price)?

In [12]:
# Create a list of all opening and close prices
close_idx = stock_json['dataset']['column_names'].index('Close')

open_close = [(x[open_idx],x[close_idx]) for x in data_points]
open_close

[(51.76, 51.76),
 (51.65, 51.6),
 (51.45, 51.82),
 (51.05, 51.32),
 (51.16, 51.4),
 (51.88, 51.27),
 (52.73, 51.66),
 (52.37, 52.62),
 (52.7, 52.01),
 (53.11, 52.67),
 (52.64, 53.09),
 (52.29, 52.43),
 (52.28, 52.14),
 (51.5, 52.12),
 (50.89, 51.47),
 (50.8, 50.89),
 (51.21, 51.25),
 (49.5, 51.14),
 (49.52, 49.86),
 (48.64, 49.7),
 (49.64, 48.75),
 (49.09, 49.25),
 (49.13, 49.2),
 (49.11, 49.11),
 (48.8, 49.2),
 (48.4, 48.8),
 (47.25, 48.39),
 (46.57, 47.04),
 (47.03, 46.84),
 (47.09, 47.03),
 (47.98, 47.05),
 (48.4, 48.0),
 (48.38, 48.34),
 (47.3, 48.34),
 (47.65, 47.21),
 (46.42, 47.47),
 (46.16, 46.26),
 (45.81, 45.99),
 (45.0, 45.97),
 (45.88, 45.27),
 (46.29, 46.04),
 (46.53, 45.76),
 (45.48, 46.41),
 (45.2, 45.41),
 (45.01, 45.0),
 (45.16, 44.85),
 (44.9, 45.0),
 (45.08, 44.87),
 (45.72, 45.0),
 (46.01, 45.77),
 (45.8, 45.96),
 (45.61, 45.55),
 (45.5, 45.4),
 (45.58, 45.43),
 (45.97, 45.29),
 (45.64, 45.84),
 (46.2, 45.74),
 (46.19, 46.0),
 (46.01, 46.05),
 (45.36, 46.11),
 (44.5

In [13]:
# Get a list of all daily swings - use absolute value
daily_swing = ['%.3f' % abs(open_close[i][0] - open_close[i][1]) for i in range(len(open_close)) if open_close[i][0] is not None and \
              open_close[i][1] is not None]
daily_swing

['0.000',
 '0.050',
 '0.370',
 '0.270',
 '0.240',
 '0.610',
 '1.070',
 '0.250',
 '0.690',
 '0.440',
 '0.450',
 '0.140',
 '0.140',
 '0.620',
 '0.580',
 '0.090',
 '0.040',
 '1.640',
 '0.340',
 '1.060',
 '0.890',
 '0.160',
 '0.070',
 '0.000',
 '0.400',
 '0.400',
 '1.140',
 '0.470',
 '0.190',
 '0.060',
 '0.930',
 '0.400',
 '0.040',
 '1.040',
 '0.440',
 '1.050',
 '0.100',
 '0.180',
 '0.970',
 '0.610',
 '0.250',
 '0.770',
 '0.930',
 '0.210',
 '0.010',
 '0.310',
 '0.100',
 '0.210',
 '0.720',
 '0.240',
 '0.160',
 '0.060',
 '0.100',
 '0.150',
 '0.680',
 '0.200',
 '0.460',
 '0.190',
 '0.040',
 '0.750',
 '0.470',
 '0.590',
 '1.560',
 '0.310',
 '0.070',
 '0.230',
 '0.580',
 '0.830',
 '0.550',
 '0.210',
 '0.230',
 '0.130',
 '0.170',
 '0.040',
 '0.480',
 '0.040',
 '0.030',
 '0.230',
 '0.110',
 '0.380',
 '0.220',
 '0.250',
 '0.100',
 '0.060',
 '0.140',
 '0.200',
 '0.500',
 '0.030',
 '0.110',
 '0.250',
 '0.160',
 '0.130',
 '0.620',
 '0.350',
 '0.250',
 '0.180',
 '0.640',
 '0.050',
 '1.690',
 '0.530',


In [14]:
# get the largest daily swing
max_swing = max(daily_swing)
max_swing

'1.690'

### 5. What was the largest change between any two days (based on Closing Price)?

In [16]:
# Create a list of all close prices
closing = [x[close_idx] for x in data_points]
closing

[51.76,
 51.6,
 51.82,
 51.32,
 51.4,
 51.27,
 51.66,
 52.62,
 52.01,
 52.67,
 53.09,
 52.43,
 52.14,
 52.12,
 51.47,
 50.89,
 51.25,
 51.14,
 49.86,
 49.7,
 48.75,
 49.25,
 49.2,
 49.11,
 49.2,
 48.8,
 48.39,
 47.04,
 46.84,
 47.03,
 47.05,
 48.0,
 48.34,
 48.34,
 47.21,
 47.47,
 46.26,
 45.99,
 45.97,
 45.27,
 46.04,
 45.76,
 46.41,
 45.41,
 45.0,
 44.85,
 45.0,
 44.87,
 45.0,
 45.77,
 45.96,
 45.55,
 45.4,
 45.43,
 45.29,
 45.84,
 45.74,
 46.0,
 46.05,
 46.11,
 44.98,
 44.17,
 43.56,
 42.04,
 42.37,
 42.07,
 42.06,
 41.46,
 41.99,
 42.44,
 42.27,
 42.42,
 42.52,
 42.45,
 42.73,
 42.85,
 42.67,
 42.77,
 42.55,
 42.62,
 42.6,
 42.41,
 41.9,
 41.94,
 41.85,
 41.91,
 42.14,
 42.69,
 42.71,
 42.71,
 42.26,
 42.41,
 42.5,
 42.05,
 42.28,
 42.3,
 41.94,
 41.68,
 41.81,
 44.37,
 44.96,
 45.07,
 44.97,
 45.56,
 45.45,
 45.3,
 44.97,
 45.25,
 45.16,
 44.82,
 44.61,
 45.44,
 45.66,
 45.57,
 45.0,
 45.6,
 45.53,
 44.95,
 44.95,
 44.2,
 44.7,
 44.53,
 44.62,
 45.19,
 44.8,
 45.75,
 45.44,
 45.2,

In [19]:
# Create a list composed of price changes between closing
daily_close_change = [ '%.3f' % abs(closing[i] - closing[i-1]) for i in range(1, len(closing))]
daily_close_change

['0.160',
 '0.220',
 '0.500',
 '0.080',
 '0.130',
 '0.390',
 '0.960',
 '0.610',
 '0.660',
 '0.420',
 '0.660',
 '0.290',
 '0.020',
 '0.650',
 '0.580',
 '0.360',
 '0.110',
 '1.280',
 '0.160',
 '0.950',
 '0.500',
 '0.050',
 '0.090',
 '0.090',
 '0.400',
 '0.410',
 '1.350',
 '0.200',
 '0.190',
 '0.020',
 '0.950',
 '0.340',
 '0.000',
 '1.130',
 '0.260',
 '1.210',
 '0.270',
 '0.020',
 '0.700',
 '0.770',
 '0.280',
 '0.650',
 '1.000',
 '0.410',
 '0.150',
 '0.150',
 '0.130',
 '0.130',
 '0.770',
 '0.190',
 '0.410',
 '0.150',
 '0.030',
 '0.140',
 '0.550',
 '0.100',
 '0.260',
 '0.050',
 '0.060',
 '1.130',
 '0.810',
 '0.610',
 '1.520',
 '0.330',
 '0.300',
 '0.010',
 '0.600',
 '0.530',
 '0.450',
 '0.170',
 '0.150',
 '0.100',
 '0.070',
 '0.280',
 '0.120',
 '0.180',
 '0.100',
 '0.220',
 '0.070',
 '0.020',
 '0.190',
 '0.510',
 '0.040',
 '0.090',
 '0.060',
 '0.230',
 '0.550',
 '0.020',
 '0.000',
 '0.450',
 '0.150',
 '0.090',
 '0.450',
 '0.230',
 '0.020',
 '0.360',
 '0.260',
 '0.130',
 '2.560',
 '0.590',


In [21]:
# get the max from the list to view the largest change
largest_change = max(daily_close_change)
largest_change

'2.560'

### 6. What was the average daily trading volume during this year?

### 7. What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)